# SIS with 4x4 matrices
Conjecture: 4x4 matrices are enough to capture SIS models exactly.

Compare with exact result on a small tree.

In [1]:
using MatrixProductBP, MatrixProductBP.Models, IndexedGraphs, Test, Random
import MatrixProductBP: nstates
import MatrixProductBP.Models: prob_xy, prob_yy, prob_y

In [2]:
T = 5

A = [0 1 0 0;
     1 0 1 1;
     0 1 0 0;
     0 1 0 0]
g = IndexedGraph(A)
N = size(A, 1)

λ = 0.5
ρ = 0.4
γ = 0.5;

In [18]:
sis = SIS(g, λ, ρ, T; γ)
bp = mpbp(sis)
rng = MersenneTwister(111)
X, _ = onesample(bp; rng)
draw_node_observations!(bp.ϕ, X, N, last_time=true; rng);

In [ ]:
p_exact, Z_exact = exact_prob(bp)
b_exact = exact_marginals(bp; p_exact)
p_ex = [[bbb[2] for bbb in bb] for bb in b_exact]

f(x,i) = x-1
r_exact = exact_autocorrelations(f, bp; p_exact)
c_exact = exact_autocovariances(f, bp; r = r_exact);

In [47]:
reset_messages!(bp);

In [48]:
svd_trunc = TruncBond(4)
iters, cb = iterate!(bp, maxiter=20; svd_trunc, showprogress=false, svd_verbose=false, 
        tol=1e-16)
println("First run with truncation to 4x4 matrices: converged after $iters iterations")

redirect_stdio(stdout="svd_log.txt") do
    iters, cb = iterate!(bp, maxiter=1; svd_trunc, showprogress=false, svd_verbose=true, 
        tol=1e-16)
end;

First run with truncation to 4x4 matrices: converged after 4 iterations


In [49]:
b_bp = beliefs(bp)
p_bp = [[bbb[2] for bbb in bb] for bb in b_bp]

f_bethe = bethe_free_energy(bp)
Z_bp = exp(-f_bethe)

r_bp = autocorrelations(f, bp)
c_bp = autocovariances(f, bp);

In [50]:
import LinearAlgebra.norm
relative_err(x,y) = norm(x-y)/norm(x)
@show relative_err(Z_exact, Z_bp)
@show relative_err(p_ex, p_bp)
@show relative_err(r_bp, r_exact)
@show relative_err(c_bp, c_exact);

relative_err(Z_exact, Z_bp) = 0.0001645055512241575
relative_err(p_ex, p_bp) = 6.294719669209858e-5
relative_err(r_bp, r_exact) = 0.00010784830338836459
relative_err(c_bp, c_exact) = 0.00010312280021259295


In [51]:
svd_trunc = TruncThresh(0.0)
iters, cb = iterate!(bp, maxiter=20; svd_trunc, showprogress=false)
println("First run with no truncation: converged after $iters iterations");

First run with no truncation: converged after 3 iterations


In [52]:
b_bp = beliefs(bp)
p_bp = [[bbb[2] for bbb in bb] for bb in b_bp]

f_bethe = bethe_free_energy(bp)
Z_bp = exp(-f_bethe)

r_bp = autocorrelations(f, bp)
c_bp = autocovariances(f, bp);

In [53]:
import LinearAlgebra.norm
relative_err(x,y) = norm(x-y)/norm(x)
@show relative_err(Z_exact, Z_bp)
@show relative_err(p_ex, p_bp)
@show relative_err(r_bp, r_exact)
@show relative_err(c_bp, c_exact);

relative_err(Z_exact, Z_bp) = 2.4507606694905323e-13
relative_err(p_ex, p_bp) = 2.505965736739499e-13
relative_err(r_bp, r_exact) = 2.5296876204590584e-13
relative_err(c_bp, c_exact) = 5.404344607862025e-13
